# Machine learning with TensorFlow

Classification of human body motion:
- walking
- sitting down
- turning right while walking
- turning left while walking

In [31]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as sg
import serial
import time
import random

## Preparing data set for training CNN

In [32]:
SAMPLE_NUMS = 260  # sec
SAMPLING_RATE = 80.0  # Hz
GYRO_RESOLUTION = 250.0 / 32768.0
ACCEL_RESOLUTION = 2.0 / 32768.0

FILE0 = './20180821_walking_straight.csv'
FILE1 = './20180821_sitting_down.csv'
FILE2 = './20180821_turning_left.csv'
FILE3 = './20180821_turning_right.csv'

TIME_INTERVAL = 260.0 / SAMPLING_RATE  # sec

MEASUREMENTS = 8
RECORDS = 260

In [33]:
gres = lambda v: v * GYRO_RESOLUTION
ares = lambda v: v * ACCEL_RESOLUTION
to_time = lambda v: v / SAMPLING_RATE

def conv(df):
    df[['gx', 'gy', 'gz']] = df[['gx', 'gy', 'gz']].apply(gres)
    df[['ax', 'ay', 'az']] = df[['ax', 'ay', 'az']].apply(ares)
    CUTOFF = 10.0
    b, a = sg.butter(5, CUTOFF/SAMPLING_RATE, btype='low')
    df[['ax', 'ay', 'az']] = df[['ax', 'ay', 'az']].apply(lambda row: sg.lfilter(b, a, row))
    CUTOFF = 10.0
    b, a = sg.butter(5, CUTOFF/SAMPLING_RATE, btype='low')
    df[['gx', 'gy', 'gz']] = df[['gx', 'gy', 'gz']].apply(lambda row: sg.lfilter(b, a, row))
    df[['cnt']] = df[['cnt']].apply(to_time)
    df.set_index('cnt', drop=True, inplace=True)
    # measurements = df.tail(1).iloc[0,0] + 1
    # return measurements

In [34]:
df0 = pd.read_csv(FILE0, dtype=np.int16)
df1 = pd.read_csv(FILE1, dtype=np.int16)
df2 = pd.read_csv(FILE2, dtype=np.int16)
df3 = pd.read_csv(FILE3, dtype=np.int16)

conv(df0)
conv(df1)
conv(df2)
conv(df3)

In [35]:
df0['label'] = np.full(RECORDS*MEASUREMENTS, 0)
df1['label'] = np.full(RECORDS*MEASUREMENTS, 1)
df2['label'] = np.full(RECORDS*MEASUREMENTS, 2)
df3['label'] = np.full(RECORDS*MEASUREMENTS, 2)

df_set, df0set, df1set, df2set, df3set = [], [], [], [], []
#df_set, df0set, df1set, df2set = [], [], [], []
#df_set, df0set, df1set = [], [], []
for i in range(MEASUREMENTS):
    df0set.append([df0[df0['id'] == i], [1,0,0,0]])
    df1set.append([df1[df1['id'] == i], [0,1,0,0]])
    df2set.append([df2[df2['id'] == i], [0,0,1,0]])
    df3set.append([df3[df3['id'] == i], [0,0,0,1]])

df_set.extend(df0set)
df_set.extend(df1set)
df_set.extend(df2set)
df_set.extend(df3set)

random.shuffle(df_set)

In [36]:
# use accel x axis and gyro z axis values for classification of human body motion
train_x = []
train_t = []
for df, label in df_set:
    values = np.concatenate((df['ax'].values, df['ay'].values, df['az'].values,
                             df['gx'].values, df['gy'].values, df['gz'].values),
                            axis=None)
    train_x.append(values)
    train_t.append(label)
#train_x

## TensorFlow

In [37]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20180821)
tf.set_random_seed(20180821)

In [38]:
num_samples = SAMPLE_NUMS * 6
num_units = 16
num_classify = 4

x = tf.placeholder(tf.float32, [None, num_samples])

w1 = tf.Variable(tf.truncated_normal([num_samples, num_units]))
b1 = tf.Variable(tf.zeros([num_units]))
hidden1 = tf.nn.tanh(tf.matmul(x, w1) + b1)

w0 = tf.Variable(tf.zeros([num_units, num_classify]))
b0 = tf.Variable(tf.zeros([num_classify]))
p = tf.nn.softmax(tf.matmul(hidden1, w0) + b0)

In [39]:
t = tf.placeholder(tf.float32, [None, num_classify])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer().minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [40]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [41]:
i = 0
for _ in range(1000):
    i += 1
    sess.run(train_step, feed_dict={x:train_x, t:train_t})
    if i % 100 == 0:
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={x:train_x, t: train_t})
        print('Step: {}, Loss: {}, Accuracy: {}'.format(i, loss_val, acc_val))

Step: 100, Loss: 22.8950138092041, Accuracy: 0.9375
Step: 200, Loss: 14.763388633728027, Accuracy: 0.9375
Step: 300, Loss: 10.783812522888184, Accuracy: 0.96875
Step: 400, Loss: 8.331635475158691, Accuracy: 1.0
Step: 500, Loss: 6.659186363220215, Accuracy: 1.0
Step: 600, Loss: 5.4530439376831055, Accuracy: 1.0
Step: 700, Loss: 4.550124168395996, Accuracy: 1.0
Step: 800, Loss: 3.85486102104187, Accuracy: 1.0
Step: 900, Loss: 3.307219982147217, Accuracy: 1.0
Step: 1000, Loss: 2.867664337158203, Accuracy: 1.0


In [42]:
p_test = sess.run(p, feed_dict={x:train_x})
p_test

array([[7.3863441e-01, 1.4137013e-01, 7.1749747e-02, 4.8245762e-02],
       [9.0220571e-02, 1.5913935e-02, 8.7194520e-01, 2.1920275e-02],
       [2.4960682e-02, 2.1515802e-02, 9.5350909e-01, 1.4491593e-05],
       [1.1360802e-01, 1.1436796e-02, 1.1609067e-03, 8.7379420e-01],
       [8.3643097e-01, 1.3018120e-02, 1.4773530e-03, 1.4907362e-01],
       [5.4507036e-02, 7.3308852e-03, 9.3810415e-01, 5.7910005e-05],
       [9.9289171e-02, 8.5185558e-01, 4.5609500e-02, 3.2456906e-03],
       [5.0328404e-02, 7.5505916e-03, 7.7807467e-04, 9.4134295e-01],
       [3.7081975e-02, 5.2375104e-03, 9.5766115e-01, 1.9418490e-05],
       [3.7081975e-02, 5.2375104e-03, 9.5766115e-01, 1.9418490e-05],
       [1.3779462e-03, 9.7538322e-01, 1.0149395e-02, 1.3089452e-02],
       [2.1676442e-02, 1.0426490e-03, 5.0926064e-05, 9.7723001e-01],
       [3.5040584e-04, 9.8836190e-01, 8.5143717e-03, 2.7732803e-03],
       [7.3999397e-02, 9.0090483e-01, 2.1517053e-02, 3.5786885e-03],
       [1.2467082e-01, 1.5539097e-

In [43]:
train_t

[[1, 0, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [1, 0, 0, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 0, 1],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 0, 1],
 [1, 0, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [1, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 1, 0, 0]]

In [44]:
sess.close()